# Questrade API Exploration Notebook

This notebook explores the Questrade API functionality provided in the `qt_api.py` module. We'll examine key functions for accessing account information, market data, and trade execution.

## Setup and Initialization

In [2]:
import sys
import os

# Get the current notebook's directory
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
# Get the project root (two levels up from the notebook)
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
# Add the project root to Python path
sys.path.append(project_root)
print(f"Project root: {project_root}")

In [ ]:
import sys
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Import the Questrade API module
import src.qt_api.qt_api as qt

# Set some display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


## Authentication

First, we need to authenticate with the Questrade API. The module handles token management automatically.


In [ ]:
# Get or refresh the authentication token
token_data = qt.get_questrade_token()
print("Authentication successful!")
print(f"API Server: {token_data['api_server']}")

## Account Information

Let's explore the account information available through the API.

### Retrieve Account List

In [ ]:
# Get all accounts
accounts = qt.get_questrade_accounts()
print(f"Found {len(accounts['accounts'])} accounts")

# Display account details in a DataFrame
account_df = pd.DataFrame(accounts['accounts'])
display(account_df)

# Store the primary account number for later use
if len(accounts['accounts']) > 0:
    primary_account = next((acc['number'] for acc in accounts['accounts'] if acc.get('isPrimary', False)), 
                          accounts['accounts'][0]['number'])
    print(f"Primary account: {primary_account}")

### Retrieve Account Balances

In [ ]:
# Get balances for the primary account
balances = qt.get_questrade_balances(primary_account)

# Display currency balances
currency_balances = pd.DataFrame(balances['perCurrencyBalances'])
print("Currency Balances:")
display(currency_balances)

# Display combined balances
combined_balances = pd.DataFrame(balances['combinedBalances'])
print("\nCombined Balances:")
display(combined_balances)

### Retrieve Account Positions

In [ ]:
# Get positions for the primary account
positions = qt.get_questrade_positions(account_df['number'][1])

# Get account type and number for the title
account_type = account_df['type'][1]
account_number = account_df['number'][1]

# Convert to DataFrame and display
if positions['positions']:
    positions_df = pd.DataFrame(positions['positions'])
    
    # Calculate market value percentage
    positions_df['marketValue_pct'] = positions_df['currentMarketValue'] / positions_df['currentMarketValue'].sum() * 100
    
    # Check if 'openPnL' exists in the dataframe before calculating profit/loss percentage
    if 'openPnL' in positions_df.columns:
        positions_df['profit_loss_pct'] = positions_df['openPnL'] / positions_df['totalCost'] * 100
        profit_cols = ['openPnL', 'profit_loss_pct']
    else:
        print("Note: 'openPnL' field not available in the API response")
        profit_cols = []
    
    # Sort by market value
    positions_df = positions_df.sort_values('currentMarketValue', ascending=False)
    
    # Display positions with important columns first
    cols = ['symbol', 'currentMarketValue', 'marketValue_pct', 'openQuantity', 
            'currentPrice', 'averageEntryPrice'] + profit_cols + ['totalCost']
    display(positions_df[cols])
    
    # Create a pie chart of position allocation
    plt.figure(figsize=(10, 8))
    plt.pie(positions_df['currentMarketValue'], labels=positions_df['symbol'], autopct='%1.1f%%')
    plt.title(f'Portfolio Allocation by Market Value - {account_type} {account_number}')
    plt.show()
else:
    print("No positions found in the account.")

### Retrieve Account Activities

In [ ]:
# Get account activities for the last 30 days


days_back = 30
start_time = (datetime.datetime.now() - datetime.timedelta(days=days_back)).replace(
    hour=0, minute=0, second=0, microsecond=0).astimezone().isoformat()
end_time = datetime.datetime.now().astimezone().isoformat()

activities = qt.get_account_activities(account_df['number'][1], start_time, end_time)

if activities['activities']:
    # Convert to DataFrame
    activities_df = pd.DataFrame(activities['activities'])
    
    # Convert date columns to datetime
    date_cols = [col for col in activities_df.columns if 'Date' in col]
    for col in date_cols:
        activities_df[col] = pd.to_datetime(activities_df[col])
    
    # Sort by transaction date
    activities_df = activities_df.sort_values('transactionDate', ascending=False)
    
    # Display activities
    display(activities_df.head(10))
    
    # Summary of activity types
    activity_types = activities_df['type'].value_counts()
    print("\nActivity Types Summary:")
    display(activity_types)
    
    # Plot activity types
    plt.figure(figsize=(12, 6))
    activity_types.plot(kind='bar')
    plt.title('Account Activity Types (Last 30 Days)')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()
else:
    print("No activities found for the selected time period.")


### Retrieve Order History

In [ ]:
# Get order history for the last 30 days
orders = qt.get_account_orders(account_df['number'][1], start_time, end_time)

if orders['orders']:
    # Convert to DataFrame
    orders_df = pd.DataFrame(orders['orders'])
    
    # Convert date columns to datetime
    time_cols = [col for col in orders_df.columns if 'Time' in col]
    for col in time_cols:
        orders_df[col] = pd.to_datetime(orders_df[col])
    
    # Sort by creation time
    orders_df = orders_df.sort_values('creationTime', ascending=False)
    
    # Display orders with selected columns
    cols = ['symbol', 'side', 'totalQuantity', 'filledQuantity', 'limitPrice', 
            'state', 'orderType', 'creationTime']
    display(orders_df[cols].head(10))
    
    # Order state summary
    order_states = orders_df['state'].value_counts()
    print("\nOrder State Summary:")
    display(order_states)
    
    # Plot order states
    plt.figure(figsize=(10, 6))
    order_states.plot(kind='bar', color='teal')
    plt.title('Order States (Last 30 Days)')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()
else:
    print("No orders found for the selected time period.")


## Market Data

Now let's explore market data functions.

### Symbol Search

In [ ]:
# Search for symbols
search_term = "AAPL"  # Try different search terms like "TSLA", "MSFT", etc.
search_results = qt.search_symbols(search_term)

if search_results['symbols']:
    search_df = pd.DataFrame(search_results['symbols'])
    display(search_df)
    
    # Store a symbol ID for later use
    sample_symbol_id = search_df.iloc[0]['symbolId']
    sample_symbol = search_df.iloc[0]['symbol']
    print(f"Selected symbol for further analysis: {sample_symbol} (ID: {sample_symbol_id})")
else:
    print(f"No results found for '{search_term}'")


### Symbol Details

In [ ]:
# Get detailed information about the selected symbol
symbol_details = qt.get_symbol_details(symbol_id=sample_symbol_id)

if symbol_details['symbols']:
    # Display as a transposed DataFrame for better readability
    details_df = pd.DataFrame(symbol_details['symbols']).iloc[0]
    display(pd.DataFrame(details_df).rename(columns={0: 'Value'}))
else:
    print(f"No details found for symbol ID {sample_symbol_id}")

### Market Quotes

In [ ]:
# Get real-time quote for the selected symbol
quote = qt.get_market_quote(symbol_id=sample_symbol_id)

if quote['quotes']:
    quote_data = quote['quotes'][0]
    
    # Create a more readable display of the quote information
    quote_df = pd.Series(quote_data)
    display(pd.DataFrame(quote_df).rename(columns={0: 'Value'}))
    
    # Show bid-ask spread
    if 'bidPrice' in quote_data and 'askPrice' in quote_data and quote_data['bidPrice'] is not None and quote_data['askPrice'] is not None:
        spread = quote_data['askPrice'] - quote_data['bidPrice']
        spread_pct = (spread / quote_data['askPrice']) * 100
        print(f"Bid-Ask Spread: ${spread:.2f} ({spread_pct:.2f}%)")
    else:
        print("Bid-Ask Spread: Not available (bid or ask prices are None)")
else:
    print(f"No quote data found for symbol ID {sample_symbol_id}")

### Historical Candles


In [ ]:
# Get historical candle data for the selected symbol
# For the last 30 days with daily candles
days_back = 30
start_time = (datetime.datetime.now() - datetime.timedelta(days=days_back)).replace(
    hour=0, minute=0, second=0, microsecond=0).astimezone().isoformat()
end_time = datetime.datetime.now().astimezone().isoformat()

candles = qt.get_candles(sample_symbol_id, start_time, end_time, interval="OneHour")

if candles['candles']:
    candles_df = pd.DataFrame(candles['candles'])
    
    # Convert start/end to datetime with utc=True to avoid FutureWarning
    candles_df['start'] = pd.to_datetime(candles_df['start'], utc=True)
    candles_df['end'] = pd.to_datetime(candles_df['end'], utc=True)
    
    # Set start as index
    candles_df.set_index('start', inplace=True)
    
    display(candles_df.head())
    
    # Plot the candlestick data
    plt.figure(figsize=(12, 6))
    
    # Plot OHLC
    plt.plot(candles_df.index, candles_df['close'], label='Close Price')
    
    # Add volume as a bar chart on a secondary axis
    ax1 = plt.gca()
    ax2 = ax1.twinx()
    ax2.bar(candles_df.index, candles_df['volume'], alpha=0.3, color='gray', label='Volume')
    
    plt.title(f'{sample_symbol} Price History (Last {days_back} Days)')
    ax1.set_ylabel('Price')
    ax2.set_ylabel('Volume')
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Calculate and display some basic statistics
    returns = candles_df['close'].pct_change().dropna()
    stats = {
        'Mean Daily Return': returns.mean() * 100,
        'Std Dev of Daily Returns': returns.std() * 100,
        'Max Daily Gain': returns.max() * 100,
        'Max Daily Loss': returns.min() * 100,
        'Total Return (Period)': ((candles_df['close'].iloc[-1] / candles_df['close'].iloc[0]) - 1) * 100
    }
    
    stats_df = pd.Series(stats, name='Value')
    stats_df = pd.DataFrame(stats_df)
    stats_df['Value'] = stats_df['Value'].map('{:.2f}%'.format)
    display(stats_df)
else:
    print(f"No candle data found for symbol ID {sample_symbol_id}")


## Market Information


In [ ]:
# Get information about available markets
markets = qt.get_markets()

if markets['markets']:
    markets_df = pd.DataFrame(markets['markets'])
    
    # Print available columns to debug
    print("Available columns:", markets_df.columns.tolist())
    
    # Display main market information - select only columns that exist
    columns_to_display = ['name', 'extendedStartTime', 'startTime', 'endTime', 'extendedEndTime']
    # Add currency if it exists
    if 'currency' in markets_df.columns:
        columns_to_display.append('currency')
    
    simple_markets_df = markets_df[columns_to_display]
    display(simple_markets_df)
    
    # Show market hours visually
    market_hours = []
    for _, market in simple_markets_df.iterrows():
        # Extract hours for visualization
        ext_start = pd.to_datetime(market['extendedStartTime']).strftime('%H:%M')
        reg_start = pd.to_datetime(market['startTime']).strftime('%H:%M')
        reg_end = pd.to_datetime(market['endTime']).strftime('%H:%M')
        ext_end = pd.to_datetime(market['extendedEndTime']).strftime('%H:%M')
        
        market_data = {
            'Market': market['name'],
            'Pre-Market': f"{ext_start} - {reg_start}",
            'Regular Hours': f"{reg_start} - {reg_end}",
            'After Hours': f"{reg_end} - {ext_end}"
        }
        
        # Add currency if it exists
        if 'currency' in market:
            market_data['Currency'] = market['currency']
        
        market_hours.append(market_data)
    
    hours_df = pd.DataFrame(market_hours)
    display(hours_df)
else:
    print("No market information available")


## Conclusion

This notebook has demonstrated the key functionality of the Questrade API, including:

1. Authentication and token management
2. Account information retrieval (balances, positions, activities)
3. Market data access (symbol search, quotes, historical data)
4. Trade history and order information

The API provides comprehensive access to Questrade's trading platform, allowing for portfolio analysis, market research, and potentially automated trading strategies.